### Prerequisites

1. Complete `foundation/02-rag-postgresql-persistent.ipynb` to generate and store embeddings
2. Ensure PostgreSQL is running with your embeddings registered
3. Install dependencies: `pip install ollama psycopg2-binary`

**Note:** This notebook is now integrated into `evaluation-lab/05-supplemental-embedding-analysis.ipynb` for a more complete evaluation workflow. Consider using that version instead for the full context of embedding analysis within the evaluation framework.

In [ ]:
import ollama
import psycopg2
from psycopg2.extras import execute_values

### Configuration

In [ ]:
# PostgreSQL connection
POSTGRES_CONFIG = {
    'host': 'localhost',
    'port': 5432,
    'database': 'rag_db',
    'user': 'postgres',
    'password': 'postgres',
}

# Models
EMBEDDING_MODEL = 'hf.co/CompendiumLabs/bge-base-en-v1.5-gguf'
LANGUAGE_MODEL = 'hf.co/bartowski/Llama-3.2-1B-Instruct-GGUF'

# Embedding model alias (must match the main notebook)
EMBEDDING_MODEL_ALIAS = 'bge_base_en_v1.5'

### Load Helper Classes

In [ ]:
# Copy the PostgreSQLVectorDB class from the main notebook
# Or import it if you've made it a shared module

class PostgreSQLVectorDB:
    """Helper class to manage embeddings in PostgreSQL with pgvector."""
    
    def __init__(self, config, table_name):
        self.config = config
        self.table_name = table_name
        self.conn = None
        self.connect()
        self.setup_table()
    
    def connect(self):
        try:
            self.conn = psycopg2.connect(
                host=self.config['host'],
                port=self.config['port'],
                database=self.config['database'],
                user=self.config['user'],
                password=self.config['password']
            )
            print(f'✓ Connected to PostgreSQL at {self.config["host"]}:{self.config["port"]}')
        except psycopg2.OperationalError as e:
            print(f'✗ Failed to connect to PostgreSQL: {e}')
            raise
    
    def setup_table(self):
        with self.conn.cursor() as cur:
            # Create extension
            cur.execute('CREATE EXTENSION IF NOT EXISTS vector')
            
            # Create table if it doesn't exist
            cur.execute(f'''
                CREATE TABLE IF NOT EXISTS {self.table_name} (
                    id SERIAL PRIMARY KEY,
                    chunk_text TEXT NOT NULL,
                    embedding vector(768),
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                )
            ''')
            
            # Create index
            index_name = f'{self.table_name}_embedding_idx'
            cur.execute(f'''
                CREATE INDEX IF NOT EXISTS {index_name}
                ON {self.table_name} USING hnsw (embedding vector_cosine_ops)
            ''')
            
            self.conn.commit()
            print(f'✓ Table "{self.table_name}" ready')
    
    def get_chunk_count(self):
        with self.conn.cursor() as cur:
            cur.execute(f'SELECT COUNT(*) FROM {self.table_name}')
            return cur.fetchone()[0]
    
    def similarity_search(self, query_embedding, top_n=3):
        with self.conn.cursor() as cur:
            cur.execute(f'''
                SELECT chunk_text, 
                       1 - (embedding <=> %s::vector) as similarity
                FROM {self.table_name}
                ORDER BY embedding <=> %s::vector
                LIMIT %s
            ''', (query_embedding, query_embedding, top_n))
            
            results = cur.fetchall()
            return [(chunk, score) for chunk, score in results]
    
    def close(self):
        if self.conn:
            self.conn.close()

### Load Embeddings

In [ ]:
# Connect to the stored embeddings
table_name = f'embeddings_{EMBEDDING_MODEL_ALIAS.replace(".", "_")}'
db = PostgreSQLVectorDB(POSTGRES_CONFIG, table_name)

count = db.get_chunk_count()
print(f'\n✓ Loaded {count} embeddings from database')

## Experiment Ideas

### 1. Analyze Query Performance

Test how well different types of queries retrieve relevant chunks:

In [ ]:
def analyze_retrieval(query, top_n=5):
    """Test retrieval quality for a query."""
    query_embedding = ollama.embed(model=EMBEDDING_MODEL, input=query)['embeddings'][0]
    results = db.similarity_search(query_embedding, top_n=top_n)
    
    print(f'\nQuery: "{query}"')
    print(f'\nTop {top_n} results:')
    for i, (chunk, score) in enumerate(results, 1):
        preview = chunk[:100].replace('\n', ' ') + '...'
        print(f'{i}. (similarity: {score:.4f}) {preview}')

# Test different query types
analyze_retrieval('What is photosynthesis?', top_n=3)
analyze_retrieval('Napoleon', top_n=3)
analyze_retrieval('programming', top_n=3)

### 2. Compare Embedding Models

If you have embeddings from different models stored in different tables, compare them:

```python
# After generating embeddings with a different model (e.g., 'sentence-transformers/all-MiniLM-L6-v2')
# you'll have a separate table

db1 = PostgreSQLVectorDB(POSTGRES_CONFIG, 'embeddings_bge_base_en_v1_5')
db2 = PostgreSQLVectorDB(POSTGRES_CONFIG, 'embeddings_all_minilm_l6_v2')

# Compare retrieval results
query = 'What is AI?'
query_emb1 = ollama.embed(model='model1', input=query)['embeddings'][0]
query_emb2 = ollama.embed(model='model2', input=query)['embeddings'][0]

results1 = db1.similarity_search(query_emb1, top_n=3)
results2 = db2.similarity_search(query_emb2, top_n=3)
```

### 3. Statistical Analysis

Analyze embedding statistics:

In [ ]:
import statistics

def analyze_embeddings():
    """Compute statistics about embeddings."""
    with db.conn.cursor() as cur:
        # Get embedding dimension
        cur.execute(f'SELECT embedding FROM {db.table_name} LIMIT 1')
        first_embedding = cur.fetchone()[0]
        dimension = len(first_embedding)
        
        print(f'Embedding dimension: {dimension}')
        
        # Calculate average pairwise similarity (optional - can be slow for large datasets)
        # This shows how similar chunks are to each other on average
        # For demonstration, we'll just report the count and dimension
        
        print(f'Total chunks: {db.get_chunk_count()}')

analyze_embeddings()

### 4. Debug Retrieval Quality

Identify queries that retrieve poor results:

In [ ]:
def test_retrieval_quality(queries):
    """Test a batch of queries and identify low-quality retrievals."""
    results = []
    
    for query in queries:
        query_embedding = ollama.embed(model=EMBEDDING_MODEL, input=query)['embeddings'][0]
        retrieved = db.similarity_search(query_embedding, top_n=1)
        
        if retrieved:
            chunk, score = retrieved[0]
            results.append({
                'query': query,
                'top_score': score,
                'preview': chunk[:50] + '...'
            })
    
    # Sort by quality
    results.sort(key=lambda x: x['top_score'])
    
    print('\nRetrieval Quality (sorted by score):')
    for r in results:
        print(f"{r['top_score']:.4f} | {r['query']:<30} | {r['preview']}")

# Test with various queries
test_queries = [
    'What is water?',
    'Tell me about plants',
    'How do computers work?',
    'What is mathematics?',
]

test_retrieval_quality(test_queries)

## Cleanup

Close database connection when done:

In [ ]:
# db.close()